# Understanding the Transfer Learning Conceptually

#### Importing the VGG16 Model

In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [2]:
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 
# But for a faster way, one can also re-size the image to (64,64)
# img_rows = 64
# img_cols = 64

#### Loading the VGG16 Model

Whenever we are loading the model, it is good practise to give the input shape of the image there!

In [3]:
model = VGG16(weights = 'imagenet',  
                 input_shape = (img_rows, img_cols, 3))

# Weights are downloaded automatically when instantiating a model. 
# They are stored at ~/.keras/models/.

Now our weights are downloaded automatically while instantiating a model. They are stored at ~/.keras/models/.

#### Inspecting each layer

In [4]:
model.layers

Now we just want the name of the layer. So, here are the steps to do so:

In [5]:
model.layers[0]

In [6]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [7]:
model.layers[0].__class__

keras.engine.input_layer.InputLayer

In [8]:
model.layers[0].__class__.__name__

'InputLayer'

As we can see above, this is how we retrieve our name of the layers. 

In [9]:
model.layers[1].__class__.__name__

'Conv2D'

Now, we want to see that can which layers are the one that we can train or not. So,

In [10]:
model.layers[0].trainable

False

In [11]:
model.layers[1].trainable

True

In [12]:
model.layers[2].trainable

True

This means that by default vgg16 layer is fixed or say that by default it is not trainable. 

For transfer learning we've to freeze all the layers.

In [13]:
for layers in model.layers:
    layers.trainable = False

In [14]:
#Print our layers
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False
19 Flatten False
20 Dense False
21 Dense False
22 Dense False


Now all of our layers are freezed!

Now we can add our layers, but by using 'add' it will give us an error!

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model

In [16]:
model.add(Dense(1024, activation = 'relu'))

AttributeError: 'Model' object has no attribute 'add'

In [17]:
model.output

<tf.Tensor 'predictions/Softmax:0' shape=(None, 1000) dtype=float32>

Since our last layer has 'Softmax' function. That means that it is the last layer which gives us the output. So, we can't directly add layers to it. Therefore, to add a layer first we've to go for 'include_top = False' which will exclude the 3 fully connected layers at the top of the network.

In [18]:
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (224, 224, 3))

* include_top: It is used to whether to include the 3 fully-connected layers at the top of the network. Here since we're doing transfer learning, we don't need it.

Now we can add our layers.

In [19]:
model.add(Dense(1024, activation = 'relu'))

AttributeError: 'Model' object has no attribute 'add'

In [20]:
model.output

<tf.Tensor 'block5_pool_1/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

Now to add model:

We can now use the Keras function API to add a new Flatten layer after the last pooling layer in the VGG16 model, then define a new classifier model with a Dense fully connected layer and an output layer that will predict the probability for 5 classes.

In [21]:
#get extracted features or add new classifier layers
bottom_model = model.output
bottom_model = Dense(1024, activation = 'relu')(bottom_model)

In [22]:
bottom_model

<tf.Tensor 'dense_1/Relu:0' shape=(None, 7, 7, 1024) dtype=float32>

In [23]:
bottom_model = Dense(516, activation = 'relu')(bottom_model)

Now we'll define our new model:

In [31]:
model = Model(inputs = model.input, outputs = bottom_model)

In [25]:
model.summary

<bound method Network.summary of <keras.engine.training.Model object at 0x000001F501FF3BC8>>

In [26]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [27]:
bottom_model = Dense(516, activation = 'relu')(bottom_model)

In [28]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [29]:
model = Model(inputs = model.input, outputs = bottom_model)

In [30]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

This is the approach taken in FaceRecog_VGG16_224_224 and FaceRecog_VGG16_64_64!